<a href="https://colab.research.google.com/github/kaniska/covid-19-hackathon/blob/master/social-network-mining/MedicalEntityRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import json
import sys
import spacy
import scispacy
import pandas as pd
from scispacy.umls_linking import UmlsEntityLinker


In [0]:
nlp = spacy.load("en_ner_bc5cdr_md")

In [0]:
with open(r"drive/My Drive/TweetStreams/coronavirus-tweet-id-2020-04-10-21.json", "r") as file:
    tweets_data = json.load(file)

doc = nlp(tweets_data)
drugs = []
ds = []
diseases = []
for entity in doc.ents:
    drug = entity.text
    label = entity.label_
    if label == "CHEMICAL" and drug not in drugs:
        diseases.append(get_disease(drug))
    elif label == "DISEASE" and drug not in drugs:
        ds.append(drug)
    drugs.append(drug)
if len(ds):
    diseases.append(ds)
if len(diseases) == 1:
    print(None)
else:
    return print(diseases)